<a href="https://colab.research.google.com/github/saikrishna7994/arialint-barebones/blob/master/ColabFewShotIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro

Hi, this notebook's going to show you the basics of how to use colab for GPT development.

Basically, we need to do 2 things:
- Upload our credentials
- Send prompts to the OpenAI Servers

We'll start by installing the relevant packages we need, we can talk about them later. To run a cell containing python code (for example, the one below that starts "!pip install openai"), select it and hit shift+enter or click the little "play" arrow in a circle at the upper left of the cell. You should run through each cell in order, since the cells depend on variables from previous cells, and some of those variables, like "prompt", get redefined.

In [ ]:
!pip install openai
import openai, json, pandas as pd
from google.colab import files

     |████████████████████████████████| 163kB 5.6MB/s 
  Created wheel for openai: filename=openai-0.3.0-cp36-none-any.whl size=171252 sha256=d2b450d9b568e7e2a0b921bec21a24ce6870dc723d551a2922edc74aadeaa58a
  Stored in directory: /root/.cache/pip/wheels/4c/75/b3/519250ae7ef404a76542edfbb7a290c78ebb314c5b8924f541
Successfully built openai


We'll upload our API key. I like loading up a .json file with the format:

{'key': 'API KEY GOES HERE'}

You can also just use

openai.api_key = 'KEY'

and replace the word 'KEY' with the long alphanumeric key starting with "sk-" you can find here: https://beta.openai.com/docs/api-reference/authentication but I do this here so we can share this without exposing keys by accident. After using shift+enter on the cell below, it will prompt you to select the key.json file from your directory.



In [ ]:
from google.colab import files
uploaded = files.upload()
print("done")
openai.api_key = json.load(open("key.json", "r"))["key"]

Saving key.json to key.json
done


## Making our first prompt

Making a prompt in colab is easy. We just write a string and send it to OpenAI to autocomplete.

Let's make our first string.

Simple Translation: English to French:

In [ ]:
prompt = """We translated each sentence from English to French.

English: I want to make some tacos
French:"""

In [ ]:
print(openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20))

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Je veux faire quelque tacos"
    }
  ],
  "created": 1612801502,
  "id": "cmpl-2R3dOsGI1eBBZlieLFvMoTyrTSw23",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


Great, we can see that it returned choices and stuff. We'll covering requesting multiple completions later, for now, let's just grab the completion['choices'][0]['text'] from now on. That's GPT-3's best guess at what should come next after the text we gave it in the prompt.

Now we can also give it more examples and see if it can translate into other languages:

In [ ]:
prompt = """We translated each sentence from English to other languages.

English: I want to make some tacos
French: Je veux faire des tacos

English: Where is the library?
Spanish:"""

In [ ]:
openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20)['choices'][0]['text']

' ¿Dónde está la biblioteca?'

That was pretty cool! I bet we can even do other stuff like fix spelling and grammar:



In [ ]:
prompt = """We fixed the spelling and grammar of each sentence.

Original: I wants some tacos
Correct: I want some tacos

Original: They is really nice.
Correct:"""

In [ ]:
openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20)['choices'][0]['text']

' They are really nice.'

Or change politeness while keeping roughly the same meaning:

In [ ]:
prompt = """We made everything polite.

Original: You suck
Translation: I don't think you did that well.

Original: Shut up.
Translation:"""

In [ ]:
openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20)['choices'][0]['text']

" Look, it's really not my fault."

Huh, so that didn't work with only 2 examples. If you hit shift-enter more than once on the same cell, you can see other possible results. Let's try giving it some more examples:

In [ ]:
prompt = """We made everything polite.

Original: You suck
Polite: I don't think you did that well.

Original: Shut up.
Polite: Please be quiet.

Original: Give me that.
Polite:"""

In [ ]:
openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20)['choices'][0]['text']

' I have nothing more to say to you.'

Let's check how we're doing so far: here's a challenge, how would we get a prompt to generate a translation from English to German?

In [ ]:
prompt = """YOUR CODE HERE

English: We'll cross that bridge when we come to it
German:"""

openai.Completion.create(prompt=prompt, engine='davinci', stop='\n', max_tokens=20)['choices'][0]['text']

Try adding more and more examples, rerunning both the prompt cell and then the completion cell, until it behaves the way you want. We can do style transfer as well

In [ ]:
prompt = """Author: Edgar Allen Poe
Topic: A Raven
Poem: But the Raven, sitting lonely on the placid bust, spoke only
That one word, as if his soul in that one word he did outpour.
Nothing further then he uttered—not a feather then he fluttered—
Till I scarcely more than muttered "Other friends have flown before—
On the morrow he will leave me, as my hopes have flown before."
Then the bird said "Nevermore."

Author: Walt Whitman
Topic: A Captain
Poem: O Captain! my Captain! our fearful trip is done,
The ship has weather’d every rack, the prize we sought is won,
The port is near, the bells I hear, the people all exulting,
While follow eyes the steady keel, the vessel grim and daring;

Author: Robert Frost
Topic: Walking in the woods
Poem: Whose woods these are I think I know.   
His house is in the village though;   
He will not see me stopping here   
To watch his woods fill up with snow.   

Author: Edgar Allen Poe
Topic: Why pineapples on pizza are bad
Poem:"""


In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n', 'Poem'], max_tokens=40)['choices'][0]['text']
print(r)

 But, most from that most evil pair, 
Who taught thee, monster, to devour 
The mangiest fragments of the earth 
And gods? Descend, ye shades,


In [ ]:
prompt = """Author: Edgar Allen Poe
Topic: A Raven
Poem: But the Raven, sitting lonely on the placid bust, spoke only
That one word, as if his soul in that one word he did outpour.
Nothing further then he uttered—not a feather then he fluttered—
Till I scarcely more than muttered "Other friends have flown before—
On the morrow he will leave me, as my hopes have flown before."
Then the bird said "Nevermore."

Author: Walt Whitman
Topic: A Captain
Poem: O Captain! my Captain! our fearful trip is done,
The ship has weather’d every rack, the prize we sought is won,
The port is near, the bells I hear, the people all exulting,
While follow eyes the steady keel, the vessel grim and daring;

Author: Robert Frost
Topic: Walking in the woods
Poem: Whose woods these are I think I know.   
His house is in the village though;   
He will not see me stopping here   
To watch his woods fill up with snow.   

Author: Robert Frost
Topic: Why pineapples on pizza are bad
Poem:"""


In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n', 'Poem'], max_tokens=40)['choices'][0]['text']
print(r)

 Why pineapples on pizza?
Don’t go there.
Ask Them why the ruler’s straight.
Don’t ask Them why the swallows fly. 


# Getting it to process things a bit more

So it's kind of mindlessly auto-completing so far. We can give it some ideas of what to do in non-normal circumstances:

Nonsense Detections

In [ ]:
prompt = """Q: What is 1+1?
A: 2

Q: What is the capital of France?
A: Paris

Q: What is the fooza of Spain?
A:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n'], max_tokens=40)['choices'][0]['text']
print(r)

 Madrid


It got the first and second questions right. How did it "know"?

There were enough examples of people asking the capital of France, or people telling the capital of France, in the material GPT-3 was trained on that "Paris" was the highest probabity word to occur next in that context. This happens for many, many different concepts. GPT-3 contains, in the implicit strengths of its neural network weights, many facts about the world and how they are related to each other, and can use those to answer questions or complete prompts in ways that make sense.

On the third question, though, this doesn't work. It has never seen the word "fooza" before. Sometimes it makes up a word. Sometimes it gives the **capital** of Spain, which isn't what we want, either. GPT-3 gets that the context is always supplying the correct answer, but since fooza isn't a word, what it comes up with will be some kind of nonsense. This is an important point: although often GPT-3 tells true facts that it has picked up from its training, it can also just make up an answer-- we call that "confabulating." GPT-3 isn't like a database, where you can rely on it always producing the same true answer that it was given in the first place. It's more like an improv actor, who has to act like they know the answer even when they don't, so just tries to fake it.

Let's give it examples of what to do if the input is nonsensical:

In [ ]:
prompt = """Q: What is 1+1?
A: 2

Q: What's a falala duobla?
A: That doesn't make sense.

Q: What is the capital of France?
A: Paris

Q: How does a famghorn eat cheese?
A: That doesn't make sense.

Q: What is the fooza of Spain?
A:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n'], max_tokens=40)['choices'][0]['text']
print(r)

 That doesn't make sense.


In [ ]:
prompt = """Q: What is 1+1?
A: 2

Q: What's a falala duobla?
A: That doesn't make sense.

Q: What is the capital of France?
A: Paris

Q: How does a famghorn eat cheese?
A: That doesn't make sense.

Q: Who was the first US president?
A:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n'], max_tokens=40)['choices'][0]['text']
print(r)

 George Washington


In [ ]:
prompt = """Q: What is 1+1?
A: 2

Q: What's a falala duobla?
A: That doesn't make sense.

Q: What is the capital of France?
A: Paris

Q: How does a famghorn eat cheese?
A: That doesn't make sense.

Q: Who was the first US king?
A:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n'], max_tokens=40)['choices'][0]['text']
print(r)

 George Washinton


OK, those initial ones don't help it get quite there, but we can see it's doing better. What examples could we add to the prompt so that it would correctly answer that the US doesn't have a king?

In [ ]:
prompt = """YOUR PROMPT HERE

Q: Who was the first US king?
A:"""

r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n\n'], max_tokens=40)['choices'][0]['text']
print(r)

# Part 3: Classification

We can give GPT examples for classifying by labels both implicit and explicit:

In [ ]:
prompt = """We labelled each sentence with whether it's about cats, dogs, or neither

Sentence: I like canines
About:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40)['choices'][0]['text']
print(r)

 Dogs


In [ ]:
prompt = """We labelled each sentence with whether it's about cats, dogs, or neither

Sentence: I like felines
About:"""

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40)['choices'][0]['text']
print(r)

 Cats


In [ ]:
prompt = """We labelled each sentence with whether it's about cats, dogs, or neither

Sentence: I like turtles
About:"""

r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40)['choices'][0]['text']
print(r)

 none


Huh, so apparently that works sometimes but don't assume it always will. More examples can help...

In [ ]:
prompt = """We gave each synopsis the appropriate TV rating

Synopsis: A dog and a cat become friends.
Rating: PG

Synopsis: Two people have a lot of sex
Rating:"""

r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40)['choices'][0]['text']
print(r)

 R


# Taking a look at the parameters we haven't been messing with

K, let's take another look at what's going on under the hood here. We'll start by going and requesting all the data we can actually get back from a completion.

Let's try... math.

In [ ]:
prompt = """1+1=2
2+4="""
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40)['choices'][0]['text']
print(r)

let's request some more data by getting the logprobs. The logprobs tell us the probabilities of each token that'll come next. We don't want the text anymore when we get this.

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5)
print(r['choices'][0])

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": {
    "text_offset": [
      10,
      11,
      11,
      11,
      11,
      11,
      11,
      11
    ],
    "token_logprobs": [
      -0.084499575,
      -0.13960928,
      -2.1977406,
      -0.06784839,
      -2.0699346,
      -0.2904232,
      -0.0246029,
      -0.059372548
    ],
    "tokens": [
      "6",
      "\n",
      "1",
      "+",
      "3",
      "=",
      "4",
      "\n"
    ],
    "top_logprobs": [
      {
        "1": -5.257019,
        "2": -3.9814146,
        "3": -4.9558797,
        "5": -3.8883283,
        "6": -0.084499575
      },
      {
        "\n": -0.13960928,
        " ": -4.9482865,
        " (": -4.020915,
        ",": -4.7713614,
        "\u00a0": -4.5964727
      },
      {
        "1": -2.1977406,
        "2": -1.9829503,
        "3": -1.8792478,
        "4": -2.4649906,
        "6": -2.3231792
      },
      {
        "*": -5.1001625,
        "+": -0.06784839,
        "/": -4.6147113,
     

Holy cow that returned a lot of stuff! Let's start by looking at the tokens that GPT actually generated there

In [ ]:
r['choices'][0]['logprobs']['tokens']

['6', '\n', '1', '+', '3', '=', '4', '\n']

Ah, so it generated '6' (what we saw in text) then '\n' and thought that we'd ask it again what 1+3=.

Why's it stopping? Well, we had the '\n' as a stop token, meaning that once GPT found the stop token it'll stop and return the results so far. It goes a little past the stop token though so we ended up with 6 tokens instead of the 1 token that we cared about.

Next, let's take a look at the tokens competing with '6' as the first token.

<OpenAIObject at 0x7f2404b6f888> JSON: {
  "1": -5.257019,
  "2": -3.9814146,
  "3": -4.9558797,
  "5": -3.8883283,
  "6": -0.084499575
}

The logprobs are the probabiltiies that each token comes next. The top logprobs are the most likely next tokens; since we asked for 5 we got the 5 most likely tokens back in this field. We can convert the logprobs back to probabilites by taking e**token.

In [ ]:
import numpy as np

In [ ]:
rslts = r['choices'][0]['logprobs']['top_logprobs'][0]
[(x, '{}%'.format(np.round(100*np.e**float(rslts[x]),2))) for x in rslts]

[('1', '0.52%'), ('2', '1.87%'), ('3', '0.7%'), ('5', '2.05%'), ('6', '91.9%')]

man it was pretty confident that '6' was next! But what about the 1+3=4 on the next line?


In [ ]:
rslts = r['choices'][0]['logprobs']['top_logprobs'][2]
[(x, '{}%'.format(np.round(100*np.e**float(rslts[x]),2))) for x in rslts]

[('1', '11.11%'),
 ('2', '13.77%'),
 ('3', '15.27%'),
 ('4', '8.5%'),
 ('6', '9.8%')]

It wasn't as confident there; there was no reason to think it would be anything so it just kind of guessed off of whatever problems it had seen like this in this context. Wait, so it wasn't taking hte most probale token each time! I guess the default temperature isn't 0.

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, temperature=0)
print(r['choices'][0]['text'])

6


In [ ]:
r['choices'][0]['logprobs']['tokens']

['6', '\n', '3', '+', '5', '=', '8', '\n']

In [ ]:
print(r['choices'][0]['text'])

6


Let's go take lower probability tokens using the temperature parameter. Temperature is a range of 0-2, were 2 is crazy non-probable next tokens. Normally temperature around .3 will result in some creativity, 1 a bit random, and above that nuts.

So we'll re-request using temperature=1

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, temperature=1)
print(r['choices'][0])

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": {
    "text_offset": [
      10,
      11,
      11,
      11,
      11,
      11,
      11,
      11
    ],
    "token_logprobs": [
      -0.08648484,
      -0.13950278,
      -5.74998,
      -4.896288,
      -3.0244753,
      -3.0125067,
      -5.5629826,
      -2.993689
    ],
    "tokens": [
      "6",
      "\n",
      "So",
      " for",
      " any",
      " given",
      " base",
      " and"
    ],
    "top_logprobs": [
      {
        "1": -5.236021,
        "2": -3.9320352,
        "3": -4.9127364,
        "5": -3.874693,
        "6": -0.08648484
      },
      {
        "\n": -0.13950278,
        " ": -4.932246,
        " (": -4.030875,
        ",": -4.7685347,
        "\u00a0": -4.527675
      },
      {
        "1": -2.1801443,
        "2": -1.978202,
        "3": -1.8764564,
        "4": -2.4651597,
        "6": -2.3318436
      },
      {
        " 2": -3.7843568,
        " far": -3.6162126,
        " the": -2.192

We can see the probabilities are still the same (GPT isn't quite deterministic)

In [ ]:
rslts = r['choices'][0]['logprobs']['top_logprobs'][0]
[(x, '{}%'.format(np.round(100*np.e**float(rslts[x]),2))) for x in rslts]

[('1', '0.53%'),
 ('2', '1.96%'),
 ('3', '0.74%'),
 ('5', '2.08%'),
 ('6', '91.71%')]

In [ ]:
r['choices'][0]['logprobs']['tokens']

['6', '\n', 'So', ' for', ' any', ' given', ' base', ' and']

woah, those weren't probable! They aren't in the top logprobs at all!

In [ ]:
rslts = r['choices'][0]['logprobs']['top_logprobs'][2]
[(x, '{}%'.format(np.round(100*np.e**float(rslts[x]),2))) for x in rslts]

[('3', '15.31%'),
 ('2', '13.83%'),
 ('1', '11.3%'),
 ('6', '9.71%'),
 ('4', '8.5%')]

So what were the probabilies of each of those? Wow, it just grabbed those low probability ones when it didn't know where to go lol

In [ ]:
rslts = r['choices'][0]['logprobs']['token_logprobs']
[(x, '{}%'.format(np.round(100*np.e**x),2)) for x in rslts]

[(-0.08648484, '92.0%'),
 (-0.13950278, '87.0%'),
 (-5.74998, '0.0%'),
 (-4.896288, '1.0%'),
 (-3.0244753, '5.0%'),
 (-3.0125067, '5.0%'),
 (-5.5629826, '0.0%'),
 (-2.993689, '5.0%')]

K, so if we want to try a bunch of different paths using temperature we can use the 'n' parameter to get more choices back so we don't need to use choices of 0 each time

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, n=4)
[print(x['logprobs']['tokens']) for x in r['choices']]

['3', '\n', '2', '=', '0', '\n', '2', '=']
['6', '\n', '4', '+', '4', '=', '8', '\n']
['6', '\n', '5', '+', '1', '=', '6', '\n']
['6', '\n', '6', '+', '5', '=', '11', '\n']


[None, None, None, None]

cool! We managed to skip out of the 90% one in 1/4 lol

We can also ask OpenAI to calculate which of those would have the highest average logprob and give just that back to us with the best_of parameter

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, best_of=4)
[print(x['logprobs']['tokens']) for x in r['choices']]

['6', '\n', '1', '+', '2', '+', '3', '+']


[None]

And we can request the n highest best_ofs (so return in order of highest -> lowest average logprob)

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, n=4, best_of=4)
[print(x['logprobs']['tokens']) for x in r['choices']]

['6', '\n', '4', 'x', '4', '\n', '4', 'x']
['→', '6', '→', '1', '+', '4', '=', '5', '\n', '4', '+', '6', '=', '→', '10', '→']
['5', '\n', '1', '+', '2', '+', '3', '+']
['bytes:\\xce', 'bytes:\\xa6', '\n', '0', '+', '2', '+', '4']


[None, None, None, None]

but we can't ask for more than the top n best_of because it only generates best_of to order them

In [ ]:
r = openai.Completion.create(prompt=prompt, engine='davinci', stop=['\n'], max_tokens=40, logprobs=5, n=10, best_of=4)
[print(x['logprobs']['tokens']) for x in r['choices']]

InvalidRequestError: ignored

Do we want to cover logit_bias / freq penalty etc here?